***GENERATED CODE FOR bidsynergyclassify PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import round
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'NoticeId', 'transformation_label': 'String Indexer'}], 'feature': 'NoticeId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '00002c41f17541e5838d75bc0bcf1149', 'max': 'ffa780b2a1f64a228d657d8bbbbe66d8', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'NoticeId'}, {'feature_label': 'NoticeId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('NoticeId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Department_Ind_Agency', 'transformation_label': 'String Indexer'}], 'feature': 'Department_Ind_Agency', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AGRICULTURE, DEPARTMENT OF', 'max': 'VETERANS AFFAIRS, DEPARTMENT OF', 'missing': '0', 'distinct': '21'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Department_Ind_Agency'}, {'feature_label': 'Department_Ind_Agency', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Department_Ind_Agency')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AwardDate', 'transformation_label': 'String Indexer'}], 'feature': 'AwardDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '3/14/2024', 'max': '4/28/2024', 'missing': '0', 'distinct': '18'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AwardDate'}, {'feature_label': 'AwardDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AwardDate')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Award$', 'threshold': 1046358.15, 'transformation_label': 'Binarizer'}], 'feature': 'Award$', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '2148745.21', 'stddev': '29359877.82', 'min': '0.0', 'max': '6.31174E8', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'Award$'}, {'feature_label': 'Award$', 'threshold': 1046358.15, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('Award$')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Awardee', 'transformation_label': 'String Indexer'}], 'feature': 'Awardee', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AERO-DYNE SUPPLY CO., INC. ', 'max': 'null ', 'missing': '0', 'distinct': '153'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Awardee'}, {'feature_label': 'Awardee', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Awardee')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryContactFullname', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryContactFullname', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '498', 'mean': '', 'stddev': '', 'min': '"Edward ""Teddy"" H. Seifert"', 'max': 'james callins', 'missing': '1', 'distinct': '228'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryContactFullname'}, {'feature_label': 'PrimaryContactFullname', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryContactFullname')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryContactEmail', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryContactEmail', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '498', 'mean': '', 'stddev': '', 'min': 'AMANDA.SWEENEY@NAVY.MIL', 'max': 'yvonne.c.stockwell.civ@us.navy.mil', 'missing': '2', 'distinct': '227'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryContactEmail'}, {'feature_label': 'PrimaryContactEmail', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryContactEmail')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Description', 'transformation_label': 'String Indexer'}], 'feature': 'Description', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': '"AMENDMENT NO. 04 This notice constitutes AMENDMENT NO. 04 to the combined synopsis/solicitation for Multi-Hoist Winch System & Air Caster System 80LARC858109Q. Offerors shall acknowledge all amendment(s) in their quote. This notice serves as the official amendment to subject synopsis/solicitation and a written amendment will not be issued. The purpose of this amendment is to extend the submission deadline for quotations. Accordingly, the due date for receipt of Offers is extended to 4:30pm EDT on Monday 08 APR 2024. Companies shall provide the information stated in the synopsis/solicitation. (End of Amendment No. 04) UPDATE01 – 03.26.2024 – Figures – A higher resolution view of Figure 20 has been included as an attachment to this solicitation. This does not extend the response date for this requirement. (End Update01) AMENDMENT NO. 03 This notice constitutes AMENDMENT NO. 03 to the combined synopsis/solicitation for Multi-Hoist Winch System & Air Caster System 80LARC858109Q. Offerors shall acknowledge all amendment(s) in their quote. This notice serves as the official amendment to subject synopsis/solicitation and a written amendment will not be issued. The purpose of this amendment is to provide responses to Offeror questions through the provision of Attachment E, as well as update the SOW and Figures. The following SOW sections and specifications are updated: 60k lbs. Capacity Winch: 4.1. 60k lbs. Capacity Winch. 4.1.1. Fleet Angle Distance. 4.1.2. Grooved Drum. 4.1.4. VFD Control System. 4.1.11. Winch Cable. 4.1.14. Two-Part Reeving. 4.1.15. 60k lbs. Capacity Sheaves with Axles. 4.1.16. 120k lbs. Capacity Synthetic Rope. 4.1.19.\xa0 Interface Hole Pattern and Outer Mold Line Dimensions. 4.1.20. Minimum Duty Cycle. 30k lbs. Capacity Winches: 4.2. 30k lbs. Capacity Winches. 4.2.4. VFD Control Systems. 4.2.10. Winch Cables. 4.2.15.\xa0 Interface Hole Pattern and Outer Mold Line Dimensions. 4.2.16. Minimum Duty Cycle. 350k lbs. Capacity Air Caster System 4.3. 350k lbs. Capacity Air Caster System. 4.3.1. Power Drive. 4.3.4. Control Console. 4.3.5. Air Hoses. 4.3.6. Spare Air Caster Load Modules. Other 5.0. Summary of Deliverables. 6.0. Documentation. Removed from SOW Removed - 120k lbs. Capacity Sheave with Base Mount. Removed - Conduit and Wiring. The following Figures are updated: Figure 6: Initial Vertical Lift of the Lander at the Gantry (Side View) Figure 7: Lander shown at Peak Pullback for Impact Dynamics Test at the Gantry (Side View) Figure 8: Lander shown at Peak Pullback for Impact Dynamics Test at the Gantry (Perspective View) Figure 9: Upper and Lower Platform Installation on Gantry Center Bent Figure 10: Upper Platform Design (Approx. 100ft long x 15ft wide x 6ft high) Figure 11: Upper Platform Design (Top & Side View) Figure 12: Lower Platform Design (Approx. 4ft wide x 15ft long & Approx. 3k lbs.) Figure 13: Ground Tower Design (Approx. 11ft wide x 23ft long x 30ft tall) Figure 16: Impact Barrier Design Figure 20: Impact Barrier to Air Bearing Interface Control Document Accordingly, Attachments A and B are updated. Please refer to the attachments for a complete copy of the updates. The due date for receipt of Offers is not extended. Companies shall provide the information stated in the synopsis/solicitation. (End of Amendment No. 03) AMENDMENT NO. 02 This notice constitutes AMENDMENT NO. 02 to the combined synopsis/solicitation for Multi-Hoist Winch System & Air Caster System 80LARC858109Q. Offerors shall acknowledge all amendment(s) in their quote. This notice serves as the official amendment to subject synopsis/solicitation and a written amendment will not be issued. The purpose of this amendment is to update the following SOW sections and specifications: 60k lbs. Capacity Winch: 4.1.1. Fleet Angle Distance.\xa0 The design of the winch shall accommodate a minimum Fleet Angle Distance of 68-feet and a single wrap drum with 160-feet\xa0 of active cable travel plus 4 dead wraps to remain on the drum.\xa0 Fleet angle shall not exceed 2.0 degrees.\xa0 Total cable length to be at least 310-feet. 4.1.2. Grooved Drum.\xa0 The design of the winch shall have a grooved drum sized for a single layer cable wrap with 160-foot active cable travel plus the 4 dead wraps. 4.1.12. Winch Cable.\xa0 The Contractor shall provide a winch cable specifically designed for the 60k lbs. capacity winch, with a length of at least 310 feet. 4.1.14. Snatch Block.\xa0 The Contractor shall equip the 60k lbs. capacity winch with a snatch block with a minimum capacity of 120k lbs., complete with sheave and swivel hook (not a swivel shackle) with a safety latch, for easy lifting sling attachment and removal. 4.1.15. Two-Part Reeving.\xa0 The 60k lbs. capacity winch shall be designed to be installed with a two-part reeving system with the dead-end securely anchored to the Gantry Center Bent Platform.\xa0 Utilizing the two-part reeving, the system shall provide 80-feet\xa0 of vertical travel for the snatch block at a capacity of 120k lbs.\xa0 The Contractor shall not be responsible for providing the dead-end anchor and mount holes for the base mount hardware for the running sheave and the dead-ended sheave on the Upper Platform on top of the Gantry Center Bent.\xa0 The sheaves with base mounts will allow for a minimum fleet angle distance of 68 feet. 4.1.16. 60k lbs. Capacity Sheaves with Base Mounts.\xa0 The Contractor shall equip the 60k lbs. capacity winch with a running sheave (60k lbs. Capacity) and a dead-ended sheave (60k lbs. Capacity) with base mounts to enable a two-part reeving system with the dead-end securely anchored to the Gantry Center Bent Platform.\xa0 Utilizing the two-part reeving, the system shall provide 80-feet\xa0 of vertical travel for the snatch block at a capacity of 120k lbs.\xa0 Each sheave and base mount shall have a minimum capacity of 60k lbs.\xa0 The Contractor shall not be responsible for providing the dead-end anchor and mounting holes for the base mount hardware for the running sheave and the dead-ended sheave on the Upper Platform on top of the Gantry Center Bent.\xa0 The sheaves with base mounts will allow for a minimum fleet angle distance of 68 feet. 4.1.17. 120k lbs. Capacity Sheave with Base Mount.\xa0\xa0 The Contractor shall supply a sheave complete with a base mount designed to support a high load capacity synthetic rope.\xa0 The equipment is essential for securely anchoring the Lander’s swing cable to the Upper Platform situated atop the Gantry Center Bent.\xa0 The provided sheave and base mount shall support a minimum load capacity of 120k lbs.\xa0 The size of the sheave and base mount shall be compatible with the synthetic rope.\xa0 The synthetic rope intended for use typically measures 2-3/4 inches in diameter, indicating the size requirement for the sheave system.\xa0 4.1.18. 120k lbs. Capacity Synthetic Rope.\xa0 The Contractor shall supply a high-capacity synthetic rope constructed with a 12-strand design and made from Ultra-High Modulus Polyethylene (UHMPE).\xa0 This rope shall be engineered to maintain a 5:1 safety factor, with a Vertical Working Load Limit (WLL) of no less than 120k lbs.\xa0 Typically, these ropes have a diameter of 2-3/4 inches.\xa0 The rope shall be 160-foot length and is designated for anchoring the Lander’s swing harness to the Upper Platform atop the Gantry Center Bent.\xa0 The rope shall be fashioned into an eye-and-eye lifting sling, complete with splice-terminated eyes at each end for robust attachment points.\xa0 Furthermore, the rope shall undergo load testing and be supplied with a load test certificate, ensuring its reliability and adherence to the specified safety factor. 30k lbs. Capacity Winches: 4.2.1. Fleet Angle Distance.\xa0 The design of the winch shall accommodate a minimum Fleet Angle Distance of 24 feet and a drum with 90-feet\xa0 of active cable travel plus 4 dead wraps to remain on the drum.\xa0 Fleet angle shall not exceed 2.0 degrees .\xa0 Total cable length to be at least 160-feet .\xa0\xa0\xa0\xa0 4.2.2. Grooved Drum.\xa0 The design of the winch shall have a grooved drum sized for a single wrap cable with a 90-foot\xa0 active travel plus the 4 dead wraps. 4.2.11. Winch Cables.\xa0 The Contractor shall provide winch cables (one for each of the three 30k lbs. capacity winches) specifically designed for the 30k lbs. capacity winches, with a length of at least 160 feet. 4.2.15. Cable Pressure Bar.\xa0\xa0 The design of the 30k lbs. capacity winches shall incorporate a cable pressure bar to guarantee even distribution of the cable across the winch drum’s width during winding and to sustain appropriate cable tension. Accordinlgy, Attachment A has been updated. Please refer to the attachment for a complete copy of the updated SOW. The due date for receipt of Offers is not extended. Companies shall provide the information stated in the synopsis/solicitation. (End of Amendment No. 02) AMENDMENT NO. 01 This notice constitutes AMENDMENT NO. 01 to the combined synopsis/solicitation for Multi-Hoist Winch System & Air Caster System 80LARC858109Q. Offerors shall acknowledge all amendment(s) in their quote. This notice serves as the official amendment to subject synopsis/solicitation and a written amendment will not be issued. The purpose of this amendment is to establish a second site visit date of 07 MAR 2024 at 1100am EST. Interested Offerors are welcome to attend either site visit date. The schedule for site vist is amended as follows: (a) Schedule for site visit: VISIT A: Date: \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Thursday 07 MAR 2024 Time:\xa0\xa0 \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 11:00am EST Location:\xa0 \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 NASA Langley Research Center \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0Building 1297 (Gantry Facility) \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0Conference Room 203 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa04 West Bush Road \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0Hampton, VA 23681 VISIT B: Date: \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Friday 08 MAR 2024 Time:\xa0\xa0 \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 11:00am EST Location:\xa0 \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 NASA Langley Research Center \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0Building 1297 (Gantry Facility) \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0\xa0Conference Room 203 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa04 West Bush Road \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0Hampton, VA 23681 The due date for receipt of Offeror reqeust to attend the site visit is not extended. The due date for receipt of Offers is not extended. Companies shall provide the information stated in the synopsis/solicitation. (End of Amendment No. 01) This notice is a combined synopsis/solicitation for commercial items prepared in accordance with the format in FAR Subpart 12.6, as supplemented with additional information included in this notice. This announcement constitutes the only solicitation; offers are being requested and a written solicitation will not be issued. This notice is being issued as a Request for Quotation (RFQ) under solicitation number 80LARC858109Q for delivery of a multi-hoist system and an air caster system in accordance with specifications provided within the attached statement of work (SOW). A summary of the required deliverables includes various winches, air caster system, power drives, cables, sheaves, controls, snatch block, certificates of conformance, and full mechanical and electrical drawings in imperial units. This is only a summary. Please reference the SOW by attachment for a complete listing of the required deliverables. The following dates and times represent firm deadlines under this solicitation: (a) 27 FEB 2024 at 4:30pm EST – Deadline for offeror request to attend the site visit (b) 15 MAR 2024 at 4:30pm EST – Deadline for offeror questions (c) 02 APR 2024 at 4:30pm EST – Deadline for submission of quotations The provisions and clauses in the RFQ are those in effect through FAC 2024-2. This acquisition is a 100% total small business set-aside. The NAICS Code and Size Standard are 333923 and 1,250 employees, respectively. The offeror shall state in their offer their size status for this acquisition. Notice to SBA qualified Nonmanufacturers and Wholesale Suppliers: This is a Nonmanufacturer Rule (NMR) Qualifying NAICS under FAR 19.505(c). No SBA waivers apply to this solicitation waving the NMR. Accordingly, Offerors are required to provide end products from manufacturers certified as small business concerns under the NAICS. All responsible sources may submit an offer which shall be considered by the agency. Delivery to NASA Langley Research Center, Hampton VA is required by November 1, 2024, after receipt of order. Delivery shall be free on board (FOB) Destination. The selected Offeror shall work with the End-user Technical Representative (TR) for delivery point specifics after award. Offers for the items(s) described above must include, solicitation number, FOB destination to this NASA Center, proposed delivery schedule, discount/payment terms, warranty duration, taxpayer identification number (TIN), identification of any special commercial terms, and be signed by an authorized company representative. Offerors are encouraged to use the Standard Form 1449, Solicitation/Contract/Order for Commercial Items form found at URL: https://www.gsa.gov/forms-library/solicitationcontractorder-commercial-items The following clauses and provisions are incorporated; full text of the clauses and provisions may be accessed electronically at: FAR - https://www.acquisition.gov/browse/index/far NFS - https://www.hq.nasa.gov/office/procurement/regs/NFS.pdf Offerors shall provide the information required by FAR 52.212-1, Instructions to Offerors-Commercial Items (SEP 2023) which is incorporated by reference. Addenda to FAR 52.212-1 are as follows: 52.204-7. If the end product(s) offered is other than domestic end product(s) as defined in the clause 52.225-1 entitled ""Buy American Act -- Supplies', 'max': 'mTeSR1 Stem Cell Kit', 'missing': '5', 'distinct': '355'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Description'}, {'feature_label': 'Description', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Description')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run bidsynergyclassifyHooks.ipynb
try:
	#sourcePreExecutionHook()

	contractopportunitiescsv = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/ContractOpportunitiesCSV66.csv', 'filename': 'ContractOpportunitiesCSV66.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(contractopportunitiescsv)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run bidsynergyclassifyHooks.ipynb
try:
	#transformationPreExecutionHook()

	bidsynergyclassifyautofe = TransformationMain.run(contractopportunitiescsv,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "NoticeId", "transformation_label": "String Indexer"}], "feature": "NoticeId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "00002c41f17541e5838d75bc0bcf1149", "max": "ffa780b2a1f64a228d657d8bbbbe66d8", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "NoticeId"}, {"transformationsData": [{"feature_label": "Department_Ind_Agency", "transformation_label": "String Indexer"}], "feature": "Department_Ind_Agency", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AGRICULTURE, DEPARTMENT OF", "max": "VETERANS AFFAIRS, DEPARTMENT OF", "missing": "0", "distinct": "21"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Department_Ind_Agency"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NaicsCode", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "380948.48", "stddev": "143454.11", "min": "0", "max": "921190", "missing": "0"}, "updatedLabel": "NaicsCode"}, {"transformationsData": [{"feature_label": "AwardDate", "transformation_label": "String Indexer"}], "feature": "AwardDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "3/14/2024", "max": "4/28/2024", "missing": "0", "distinct": "18"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AwardDate"}, {"transformationsData": [{"feature_label": "Award$", "threshold": 1046358.15, "transformation_label": "Binarizer"}], "feature": "Award$", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2148745.21", "stddev": "29359877.82", "min": "0.0", "max": "6.31174E8", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "Award$"}, {"transformationsData": [{"feature_label": "Awardee", "transformation_label": "String Indexer"}], "feature": "Awardee", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AERO-DYNE SUPPLY CO., INC. ", "max": "null ", "missing": "0", "distinct": "153"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Awardee"}, {"transformationsData": [{"feature_label": "PrimaryContactFullname", "transformation_label": "String Indexer"}], "feature": "PrimaryContactFullname", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "498", "mean": "", "stddev": "", "min": "\"Edward \"\"Teddy\"\" H. Seifert\"", "max": "james callins", "missing": "1", "distinct": "228"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryContactFullname"}, {"transformationsData": [{"feature_label": "PrimaryContactEmail", "transformation_label": "String Indexer"}], "feature": "PrimaryContactEmail", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "498", "mean": "", "stddev": "", "min": "AMANDA.SWEENEY@NAVY.MIL", "max": "yvonne.c.stockwell.civ@us.navy.mil", "missing": "2", "distinct": "227"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryContactEmail"}, {"transformationsData": [{"feature_label": "Description", "transformation_label": "String Indexer"}], "feature": "Description", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "\"AMENDMENT NO. 04 This notice constitutes AMENDMENT NO. 04 to the combined synopsis/solicitation for Multi-Hoist Winch System & Air Caster System 80LARC858109Q. Offerors shall acknowledge all amendment(s) in their quote. This notice serves as the official amendment to subject synopsis/solicitation and a written amendment will not be issued. The purpose of this amendment is to extend the submission deadline for quotations. Accordingly, the due date for receipt of Offers is extended to 4:30pm EDT on Monday 08 APR 2024. Companies shall provide the information stated in the synopsis/solicitation. (End of Amendment No. 04) UPDATE01 \u2013 03.26.2024 \u2013 Figures \u2013 A higher resolution view of Figure 20 has been included as an attachment to this solicitation. This does not extend the response date for this requirement. (End Update01) AMENDMENT NO. 03 This notice constitutes AMENDMENT NO. 03 to the combined synopsis/solicitation for Multi-Hoist Winch System & Air Caster System 80LARC858109Q. Offerors shall acknowledge all amendment(s) in their quote. This notice serves as the official amendment to subject synopsis/solicitation and a written amendment will not be issued. The purpose of this amendment is to provide responses to Offeror questions through the provision of Attachment E, as well as update the SOW and Figures. The following SOW sections and specifications are updated: 60k lbs. Capacity Winch: 4.1. 60k lbs. Capacity Winch. 4.1.1. Fleet Angle Distance. 4.1.2. Grooved Drum. 4.1.4. VFD Control System. 4.1.11. Winch Cable. 4.1.14. Two-Part Reeving. 4.1.15. 60k lbs. Capacity Sheaves with Axles. 4.1.16. 120k lbs. Capacity Synthetic Rope. 4.1.19.\u00a0 Interface Hole Pattern and Outer Mold Line Dimensions. 4.1.20. Minimum Duty Cycle. 30k lbs. Capacity Winches: 4.2. 30k lbs. Capacity Winches. 4.2.4. VFD Control Systems. 4.2.10. Winch Cables. 4.2.15.\u00a0 Interface Hole Pattern and Outer Mold Line Dimensions. 4.2.16. Minimum Duty Cycle. 350k lbs. Capacity Air Caster System 4.3. 350k lbs. Capacity Air Caster System. 4.3.1. Power Drive. 4.3.4. Control Console. 4.3.5. Air Hoses. 4.3.6. Spare Air Caster Load Modules. Other 5.0. Summary of Deliverables. 6.0. Documentation. Removed from SOW Removed - 120k lbs. Capacity Sheave with Base Mount. Removed - Conduit and Wiring. The following Figures are updated: Figure 6: Initial Vertical Lift of the Lander at the Gantry (Side View) Figure 7: Lander shown at Peak Pullback for Impact Dynamics Test at the Gantry (Side View) Figure 8: Lander shown at Peak Pullback for Impact Dynamics Test at the Gantry (Perspective View) Figure 9: Upper and Lower Platform Installation on Gantry Center Bent Figure 10: Upper Platform Design (Approx. 100ft long x 15ft wide x 6ft high) Figure 11: Upper Platform Design (Top & Side View) Figure 12: Lower Platform Design (Approx. 4ft wide x 15ft long & Approx. 3k lbs.) Figure 13: Ground Tower Design (Approx. 11ft wide x 23ft long x 30ft tall) Figure 16: Impact Barrier Design Figure 20: Impact Barrier to Air Bearing Interface Control Document Accordingly, Attachments A and B are updated. Please refer to the attachments for a complete copy of the updates. The due date for receipt of Offers is not extended. Companies shall provide the information stated in the synopsis/solicitation. (End of Amendment No. 03) AMENDMENT NO. 02 This notice constitutes AMENDMENT NO. 02 to the combined synopsis/solicitation for Multi-Hoist Winch System & Air Caster System 80LARC858109Q. Offerors shall acknowledge all amendment(s) in their quote. This notice serves as the official amendment to subject synopsis/solicitation and a written amendment will not be issued. The purpose of this amendment is to update the following SOW sections and specifications: 60k lbs. Capacity Winch: 4.1.1. Fleet Angle Distance.\u00a0 The design of the winch shall accommodate a minimum Fleet Angle Distance of 68-feet and a single wrap drum with 160-feet\u00a0 of active cable travel plus 4 dead wraps to remain on the drum.\u00a0 Fleet angle shall not exceed 2.0 degrees.\u00a0 Total cable length to be at least 310-feet. 4.1.2. Grooved Drum.\u00a0 The design of the winch shall have a grooved drum sized for a single layer cable wrap with 160-foot active cable travel plus the 4 dead wraps. 4.1.12. Winch Cable.\u00a0 The Contractor shall provide a winch cable specifically designed for the 60k lbs. capacity winch, with a length of at least 310 feet. 4.1.14. Snatch Block.\u00a0 The Contractor shall equip the 60k lbs. capacity winch with a snatch block with a minimum capacity of 120k lbs., complete with sheave and swivel hook (not a swivel shackle) with a safety latch, for easy lifting sling attachment and removal. 4.1.15. Two-Part Reeving.\u00a0 The 60k lbs. capacity winch shall be designed to be installed with a two-part reeving system with the dead-end securely anchored to the Gantry Center Bent Platform.\u00a0 Utilizing the two-part reeving, the system shall provide 80-feet\u00a0 of vertical travel for the snatch block at a capacity of 120k lbs.\u00a0 The Contractor shall not be responsible for providing the dead-end anchor and mount holes for the base mount hardware for the running sheave and the dead-ended sheave on the Upper Platform on top of the Gantry Center Bent.\u00a0 The sheaves with base mounts will allow for a minimum fleet angle distance of 68 feet. 4.1.16. 60k lbs. Capacity Sheaves with Base Mounts.\u00a0 The Contractor shall equip the 60k lbs. capacity winch with a running sheave (60k lbs. Capacity) and a dead-ended sheave (60k lbs. Capacity) with base mounts to enable a two-part reeving system with the dead-end securely anchored to the Gantry Center Bent Platform.\u00a0 Utilizing the two-part reeving, the system shall provide 80-feet\u00a0 of vertical travel for the snatch block at a capacity of 120k lbs.\u00a0 Each sheave and base mount shall have a minimum capacity of 60k lbs.\u00a0 The Contractor shall not be responsible for providing the dead-end anchor and mounting holes for the base mount hardware for the running sheave and the dead-ended sheave on the Upper Platform on top of the Gantry Center Bent.\u00a0 The sheaves with base mounts will allow for a minimum fleet angle distance of 68 feet. 4.1.17. 120k lbs. Capacity Sheave with Base Mount.\u00a0\u00a0 The Contractor shall supply a sheave complete with a base mount designed to support a high load capacity synthetic rope.\u00a0 The equipment is essential for securely anchoring the Lander\u2019s swing cable to the Upper Platform situated atop the Gantry Center Bent.\u00a0 The provided sheave and base mount shall support a minimum load capacity of 120k lbs.\u00a0 The size of the sheave and base mount shall be compatible with the synthetic rope.\u00a0 The synthetic rope intended for use typically measures 2-3/4 inches in diameter, indicating the size requirement for the sheave system.\u00a0 4.1.18. 120k lbs. Capacity Synthetic Rope.\u00a0 The Contractor shall supply a high-capacity synthetic rope constructed with a 12-strand design and made from Ultra-High Modulus Polyethylene (UHMPE).\u00a0 This rope shall be engineered to maintain a 5:1 safety factor, with a Vertical Working Load Limit (WLL) of no less than 120k lbs.\u00a0 Typically, these ropes have a diameter of 2-3/4 inches.\u00a0 The rope shall be 160-foot length and is designated for anchoring the Lander\u2019s swing harness to the Upper Platform atop the Gantry Center Bent.\u00a0 The rope shall be fashioned into an eye-and-eye lifting sling, complete with splice-terminated eyes at each end for robust attachment points.\u00a0 Furthermore, the rope shall undergo load testing and be supplied with a load test certificate, ensuring its reliability and adherence to the specified safety factor. 30k lbs. Capacity Winches: 4.2.1. Fleet Angle Distance.\u00a0 The design of the winch shall accommodate a minimum Fleet Angle Distance of 24 feet and a drum with 90-feet\u00a0 of active cable travel plus 4 dead wraps to remain on the drum.\u00a0 Fleet angle shall not exceed 2.0 degrees .\u00a0 Total cable length to be at least 160-feet .\u00a0\u00a0\u00a0\u00a0 4.2.2. Grooved Drum.\u00a0 The design of the winch shall have a grooved drum sized for a single wrap cable with a 90-foot\u00a0 active travel plus the 4 dead wraps. 4.2.11. Winch Cables.\u00a0 The Contractor shall provide winch cables (one for each of the three 30k lbs. capacity winches) specifically designed for the 30k lbs. capacity winches, with a length of at least 160 feet. 4.2.15. Cable Pressure Bar.\u00a0\u00a0 The design of the 30k lbs. capacity winches shall incorporate a cable pressure bar to guarantee even distribution of the cable across the winch drum\u2019s width during winding and to sustain appropriate cable tension. Accordinlgy, Attachment A has been updated. Please refer to the attachment for a complete copy of the updated SOW. The due date for receipt of Offers is not extended. Companies shall provide the information stated in the synopsis/solicitation. (End of Amendment No. 02) AMENDMENT NO. 01 This notice constitutes AMENDMENT NO. 01 to the combined synopsis/solicitation for Multi-Hoist Winch System & Air Caster System 80LARC858109Q. Offerors shall acknowledge all amendment(s) in their quote. This notice serves as the official amendment to subject synopsis/solicitation and a written amendment will not be issued. The purpose of this amendment is to establish a second site visit date of 07 MAR 2024 at 1100am EST. Interested Offerors are welcome to attend either site visit date. The schedule for site vist is amended as follows: (a) Schedule for site visit: VISIT A: Date: \u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0 Thursday 07 MAR 2024 Time:\u00a0\u00a0 \u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0 11:00am EST Location:\u00a0 \u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0 NASA Langley Research Center \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0Building 1297 (Gantry Facility) \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0Conference Room 203 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a04 West Bush Road \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0Hampton, VA 23681 VISIT B: Date: \u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0 Friday 08 MAR 2024 Time:\u00a0\u00a0 \u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0 11:00am EST Location:\u00a0 \u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0 NASA Langley Research Center \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0Building 1297 (Gantry Facility) \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0\u00a0Conference Room 203 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a04 West Bush Road \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0 \u00a0Hampton, VA 23681 The due date for receipt of Offeror reqeust to attend the site visit is not extended. The due date for receipt of Offers is not extended. Companies shall provide the information stated in the synopsis/solicitation. (End of Amendment No. 01) This notice is a combined synopsis/solicitation for commercial items prepared in accordance with the format in FAR Subpart 12.6, as supplemented with additional information included in this notice. This announcement constitutes the only solicitation; offers are being requested and a written solicitation will not be issued. This notice is being issued as a Request for Quotation (RFQ) under solicitation number 80LARC858109Q for delivery of a multi-hoist system and an air caster system in accordance with specifications provided within the attached statement of work (SOW). A summary of the required deliverables includes various winches, air caster system, power drives, cables, sheaves, controls, snatch block, certificates of conformance, and full mechanical and electrical drawings in imperial units. This is only a summary. Please reference the SOW by attachment for a complete listing of the required deliverables. The following dates and times represent firm deadlines under this solicitation: (a) 27 FEB 2024 at 4:30pm EST \u2013 Deadline for offeror request to attend the site visit (b) 15 MAR 2024 at 4:30pm EST \u2013 Deadline for offeror questions (c) 02 APR 2024 at 4:30pm EST \u2013 Deadline for submission of quotations The provisions and clauses in the RFQ are those in effect through FAC 2024-2. This acquisition is a 100% total small business set-aside. The NAICS Code and Size Standard are 333923 and 1,250 employees, respectively. The offeror shall state in their offer their size status for this acquisition. Notice to SBA qualified Nonmanufacturers and Wholesale Suppliers: This is a Nonmanufacturer Rule (NMR) Qualifying NAICS under FAR 19.505(c). No SBA waivers apply to this solicitation waving the NMR. Accordingly, Offerors are required to provide end products from manufacturers certified as small business concerns under the NAICS. All responsible sources may submit an offer which shall be considered by the agency. Delivery to NASA Langley Research Center, Hampton VA is required by November 1, 2024, after receipt of order. Delivery shall be free on board (FOB) Destination. The selected Offeror shall work with the End-user Technical Representative (TR) for delivery point specifics after award. Offers for the items(s) described above must include, solicitation number, FOB destination to this NASA Center, proposed delivery schedule, discount/payment terms, warranty duration, taxpayer identification number (TIN), identification of any special commercial terms, and be signed by an authorized company representative. Offerors are encouraged to use the Standard Form 1449, Solicitation/Contract/Order for Commercial Items form found at URL: https://www.gsa.gov/forms-library/solicitationcontractorder-commercial-items The following clauses and provisions are incorporated; full text of the clauses and provisions may be accessed electronically at: FAR - https://www.acquisition.gov/browse/index/far NFS - https://www.hq.nasa.gov/office/procurement/regs/NFS.pdf Offerors shall provide the information required by FAR 52.212-1, Instructions to Offerors-Commercial Items (SEP 2023) which is incorporated by reference. Addenda to FAR 52.212-1 are as follows: 52.204-7. If the end product(s) offered is other than domestic end product(s) as defined in the clause 52.225-1 entitled \"\"Buy American Act -- Supplies", "max": "mTeSR1 Stem Cell Kit", "missing": "5", "distinct": "355"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Description"}]}))

	#transformationPostExecutionHook(bidsynergyclassifyautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run bidsynergyclassifyHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(bidsynergyclassifyautofe, ["NaicsCode", "NoticeId_stringindexer", "AwardDate_stringindexer", "Award$_binarizer", "Awardee_stringindexer", "PrimaryContactFullname_stringindexer", "PrimaryContactEmail_stringindexer", "Description_stringindexer"], "Department_Ind_Agency_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

